In [1]:
from langchain_experimental.tools import PythonAstREPLTool
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
import os
from langchain_core.prompts import format_document
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import streamlit as st
from langchain_community.output_parsers.rail_parser import GuardrailsOutputParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

llm = AzureChatOpenAI(
            deployment_name = "gpt-35-turbo-16k",
            azure_endpoint= st.secrets['OPENAI_API_ENDPOINT'],
            openai_api_type="azure",
            openai_api_version = "2023-07-01-preview"
        )

In [ ]:
from langchain_experimental.tools import PythonAstREPLTool
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd
import os
from langchain_core.prompts import format_document
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import streamlit as st
from langchain_community.output_parsers.rail_parser import GuardrailsOutputParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

llm = AzureChatOpenAI(
            deployment_name = "gpt-35-turbo-16k",
            azure_endpoint= st.secrets['OPENAI_API_ENDPOINT'],
            openai_api_type="azure",
            openai_api_version = "2023-07-01-preview"
        )

from PyPDF2 import PdfReader
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import os
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# Get your API keys from Openai, you will need to create an account.
import os
os.environ["OPENAI_API_KEY"] = "YOURAPIKEY"


# read in your pdf file
pdf_reader = PdfReader()  

# read data from the file and put them into a variable called text
text = ''
for i, page in enumerate(pdf_reader.pages):
text = page.extract_text()
if text:
text += text

text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 512,
chunk_overlap  = 32,
length_function = len,
)
texts = text_splitter.split_text(text)

# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

docsearch = FAISS.from_texts(texts, embeddings)

chain = load_qa_chain(OpenAI(), chain_type="stuff")

query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [6]:
import streamlit as st
from langchain.llms import OpenAI
from PyPDF2 import PdfReader
import pandas as pd
import os
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain

# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]

# Initialize LLM
llm = AzureChatOpenAI(
            deployment_name = "gpt-35-turbo-16k",
            azure_endpoint= st.secrets['OPENAI_API_ENDPOINT'],
            openai_api_type="azure",
            openai_api_version = "2023-07-01-preview"
        )

# Function to read and summarize PDF files
def summarize_pdf(file):
    pdf_reader = PdfReader(file)
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=32, length_function=len)
    texts = text_splitter.split_text(text)

    embeddings = OpenAIEmbeddings()
    docsearch = FAISS.from_texts(texts, embeddings)

    chain = load_qa_chain(llm, chain_type="stuff")
    query = "Summarize the content of this PDF."
    docs = docsearch.similarity_search(query)
    summary = chain.run(input_documents=docs, question=query)
    return summary

# Function to handle CSV and Excel files
def handle_csv_excel(file, file_type):
    if file_type == 'csv':
        df = pd.read_csv(file)
    elif file_type == 'excel':
        df = pd.read_excel(file)
    
    st.write(df.head())

    # Example operation on the dataframe
    summary = f"The file has {df.shape[0]} rows and {df.shape[1]} columns."
    return summary

# Streamlit file uploader
st.title("File Summarization App")
uploaded_file = st.file_uploader("Upload a PDF, CSV, or Excel file", type=["pdf", "csv", "xlsx"])

if uploaded_file is not None:
    file_type = uploaded_file.type
    if file_type == "application/pdf":
        st.write("Summarizing PDF file...")
        summary = summarize_pdf(uploaded_file)
        st.write("Summary:")
        st.write(summary)
    elif file_type == "text/csv":
        st.write("Handling CSV file...")
        summary = handle_csv_excel(uploaded_file, 'csv')
        st.write("Summary:")
        st.write(summary)
    elif file_type == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet":
        st.write("Handling Excel file...")
        summary = handle_csv_excel(uploaded_file, 'excel')
        st.write("Summary:")
        st.write(summary)
    else:
        st.write("Unsupported file type.")

